In [1]:
import pandas as pd
import numpy as np
from Decision_Trees_model.decision_trees import GBDT_classification
from Decision_Trees_model.decision_trees import RandomForest_regression
from Decision_Trees_model.decision_trees import RandomForest_classification
from Decision_Trees_model.decision_trees import DecisionTree_regression
from Decision_Trees_model.decision_trees import DecisionTree_classification
from Decision_Trees_model.decision_trees import GBDT_with_sklearn_classification
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [3]:
train = pd.read_csv('C:/Users/zheny/PycharmProjects/Machine_learning_human_activity/Datasets/har_train.csv')
test = pd.read_csv('C:/Users/zheny/PycharmProjects/Machine_learning_human_activity/Datasets/har_test.csv')
x_train = train.drop(["Activity", "subject"], axis=1).to_numpy()
y_train = train['Activity'].to_numpy()
x_test = test.drop(["Activity", "subject"], axis=1).to_numpy()
y_test = test['Activity'].to_numpy()
forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

tree_depths = [estimator.tree_.max_depth for estimator in forest.estimators_]

average_depth = sum(tree_depths) / len(tree_depths)
print("Average depth of trees:", average_depth)

0.9246691550729556
[[537   0   0   0   0   0]
 [  0 434  57   0   0   0]
 [  0  51 481   0   0   0]
 [  0   0   0 482   7   7]
 [  0   0   0  16 361  43]
 [  0   0   0  35   6 430]]
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.89      0.88      0.89       491
          STANDING       0.89      0.90      0.90       532
           WALKING       0.90      0.97      0.94       496
WALKING_DOWNSTAIRS       0.97      0.86      0.91       420
  WALKING_UPSTAIRS       0.90      0.91      0.90       471

          accuracy                           0.92      2947
         macro avg       0.93      0.92      0.92      2947
      weighted avg       0.93      0.92      0.92      2947

Average depth of trees: 18.96


In [5]:
model = GBDT_with_sklearn_classification(n_estimators=50, max_depth=8, learning_rate=0.1,
                                         subsample=0.85, min_samples_split=20)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred_train = model.predict(x_train)

print(f"Accuracy on har_test: {accuracy_score(y_test, y_pred)}"
      f"\nAccuracy on har_train: {accuracy_score(y_train, y_pred_train)}"
      f"\nConfusion matrix:\n{confusion_matrix(y_test, y_pred)}"
      f"\nClassification report:\n{classification_report(y_test, y_pred)}")

Обучение GBDT: 100%|██████████| 50/50 [23:57<00:00, 28.74s/it]


Accuracy on test: 0.9063454360366474
Accuracy on train: 0.9993199129488575
Confusion matrix:
[[537   0   0   0   0   0]
 [  0 394  94   0   0   3]
 [  0  53 478   0   0   1]
 [  2   1   0 463   9  21]
 [  5   0   3  15 372  25]
 [  1   1   0  35   7 427]]
Classification report:
                    precision    recall  f1-score   support

            LAYING       0.99      1.00      0.99       537
           SITTING       0.88      0.80      0.84       491
          STANDING       0.83      0.90      0.86       532
           WALKING       0.90      0.93      0.92       496
WALKING_DOWNSTAIRS       0.96      0.89      0.92       420
  WALKING_UPSTAIRS       0.90      0.91      0.90       471

          accuracy                           0.91      2947
         macro avg       0.91      0.90      0.91      2947
      weighted avg       0.91      0.91      0.91      2947



In [12]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

model = XGBClassifier(n_estimators=100)
model.fit(x_train, y_train_enc, eval_set=[(x_train, y_train_enc)], verbose=False)
y_pred = model.predict(x_test)
y_pred_labels = le.inverse_transform(y_pred)

print(f"Accuracy score: {accuracy_score(y_test, y_pred_labels)}")
print(f"Confusion matrix:\n{confusion_matrix(y_test, y_pred_labels)}")
print(f"Classification report:\n{classification_report(y_test, y_pred_labels)}")

Accuracy score: 0.9392602646759416
Confusion matrix:
[[537   0   0   0   0   0]
 [  0 423  65   0   0   3]
 [  0  27 505   0   0   0]
 [  0   0   0 484   8   4]
 [  0   0   0   9 383  28]
 [  0   1   0  28   6 436]]
Classification report:
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.94      0.86      0.90       491
          STANDING       0.89      0.95      0.92       532
           WALKING       0.93      0.98      0.95       496
WALKING_DOWNSTAIRS       0.96      0.91      0.94       420
  WALKING_UPSTAIRS       0.93      0.93      0.93       471

          accuracy                           0.94      2947
         macro avg       0.94      0.94      0.94      2947
      weighted avg       0.94      0.94      0.94      2947



In [4]:
model = GBDT_classification(n_estimators=50, max_depth=8, subsample=0.85,
                            learning_rate=0.05,lambda_reg=0.1, alpha_reg=0.1)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred_train = model.predict(x_train)

print(f"Количество деревьев: {10}; Глубина деревьев: {5}; lambda = {0.1}; alpha = {0.1}")
print(f"Accuracy score har_test: {accuracy_score(y_test, y_pred)}")
print(f"Accuracy score har_train: {accuracy_score(y_train, y_pred_train)}")
print(f"Confusion matrix:\n{confusion_matrix(y_test, y_pred)}")
print(f"Classification report:\n{classification_report(y_test, y_pred)}\n")

Обучение GBDT:   2%|▏         | 1/50 [08:25<6:52:54, 505.61s/it]

KeyboardInterrupt



In [10]:
estimators = [10, 15, 20, 30, 40, 60, 80, 100]
max_depths = [3, 4, 5, 6, 7, 8]
alphas = [0.1, 0.05, 0.001, 0.0001, 1]
lambdas = [0.1, 0.05, 0.001, 0.0001, 1]
for estimator in estimators:
    for depth in max_depths:
        for lam in lambdas:
            for alpha in alphas:
                model = GBDT_classification(n_estimators=estimator, max_depth=depth, subsample=0.85, learning_rate=0.05,
                                            lambda_reg=lam, alpha_reg=alpha)
                model.fit(x_train, y_train)
                y_pred = model.predict(x_test)
                y_pred_train = model.predict(x_train)

                print(f"Количество деревьев: {estimator}; Глубина деревьев: {depth}")
                print(f"Accuracy score har_test: {accuracy_score(y_test, y_pred)}")
                print(f"Accuracy score har_train: {accuracy_score(y_train, y_pred_train)}")
                print(f"Confusion matrix:\n{confusion_matrix(y_test, y_pred)}")
                print(f"Classification report:\n{classification_report(y_test, y_pred)}")

Обучение GBDT: 100%|██████████| 10/10 [27:10<00:00, 163.05s/it]


Количество деревьев: 10; Глубина деревьев: 4
Accuracy score: 0.837461825585341
Confusion matrix:
[[537   0   0   0   0   0]
 [  0 390  97   0   0   4]
 [  4  93 431   0   0   4]
 [ 14   1   2 456  20   3]
 [ 68   1   0  17 309  25]
 [ 42   0   3  67  14 345]]
Classification report:
                    precision    recall  f1-score   support

            LAYING       0.81      1.00      0.89       537
           SITTING       0.80      0.79      0.80       491
          STANDING       0.81      0.81      0.81       532
           WALKING       0.84      0.92      0.88       496
WALKING_DOWNSTAIRS       0.90      0.74      0.81       420
  WALKING_UPSTAIRS       0.91      0.73      0.81       471

          accuracy                           0.84      2947
         macro avg       0.85      0.83      0.83      2947
      weighted avg       0.84      0.84      0.84      2947




Обучение GBDT: 100%|██████████| 10/10 [42:20<00:00, 254.06s/it]


Количество деревьев: 10; Глубина деревьев: 5
Accuracy score: 0.823210044112657
Confusion matrix:
[[537   0   0   0   0   0]
 [  0 380 107   0   0   4]
 [  2  77 448   0   0   5]
 [ 16   2   1 398  20  59]
 [ 66   0   1  11 307  35]
 [ 52   1   3  57   2 356]]
Classification report:
                    precision    recall  f1-score   support

            LAYING       0.80      1.00      0.89       537
           SITTING       0.83      0.77      0.80       491
          STANDING       0.80      0.84      0.82       532
           WALKING       0.85      0.80      0.83       496
WALKING_DOWNSTAIRS       0.93      0.73      0.82       420
  WALKING_UPSTAIRS       0.78      0.76      0.77       471

          accuracy                           0.82      2947
         macro avg       0.83      0.82      0.82      2947
      weighted avg       0.83      0.82      0.82      2947




Обучение GBDT: 100%|██████████| 10/10 [59:45<00:00, 358.51s/it]


...

Обучение GBDT: 100%|██████████| 20/20 [52:41<00:00, 158.08s/it]


Количество деревьев: 20; Глубина деревьев: 4
Accuracy score: 0.8713946386155412
Confusion matrix:
[[537   0   0   0   0   0]
 [  0 400  87   0   0   4]
 [  0  81 448   0   0   3]
 [ 11   1   2 462  18   2]
 [ 27   0   1  17 349  26]
 [ 21   2   1  60  15 372]]
Classification report:
                    precision    recall  f1-score   support

            LAYING       0.90      1.00      0.95       537
           SITTING       0.83      0.81      0.82       491
          STANDING       0.83      0.84      0.84       532
           WALKING       0.86      0.93      0.89       496
WALKING_DOWNSTAIRS       0.91      0.83      0.87       420
  WALKING_UPSTAIRS       0.91      0.79      0.85       471

          accuracy                           0.87      2947
         macro avg       0.87      0.87      0.87      2947
      weighted avg       0.87      0.87      0.87      2947




Обучение GBDT: 100%|██████████| 20/20 [1:27:09<00:00, 261.46s/it]


Количество деревьев: 20; Глубина деревьев: 5
Accuracy score: 0.8527315914489311
Confusion matrix:
[[537   0   0   0   0   0]
 [  0 382 105   0   0   4]
 [  0  74 453   0   0   5]
 [ 13   1   2 413  21  46]
 [ 33   0   1   9 350  27]
 [ 30   3   1  49  10 378]]
Classification report:
                    precision    recall  f1-score   support

            LAYING       0.88      1.00      0.93       537
           SITTING       0.83      0.78      0.80       491
          STANDING       0.81      0.85      0.83       532
           WALKING       0.88      0.83      0.85       496
WALKING_DOWNSTAIRS       0.92      0.83      0.87       420
  WALKING_UPSTAIRS       0.82      0.80      0.81       471

          accuracy                           0.85      2947
         macro avg       0.85      0.85      0.85      2947
      weighted avg       0.85      0.85      0.85      2947




Обучение GBDT:  15%|█▌        | 3/20 [22:41<2:08:35, 453.86s/it]

KeyboardInterrupt

